# Train a Coqui 🐸 STT model with a Common Voice Dataset 🤖

👋 Hello and welcome

This is a copy of the official colab file that regularly gets updates:
https://github.com/coqui-ai/STT/tree/main/notebooks

You have to do three things to get this code running for your project:


*   Download the Common Voice Dataset and upload it to your Google Drive
*   create a alphabet.txt file for your language
* change a few of the paths to match your folder structure on Drive and your language code

I used this notebook with a payed Google Drive account and Colab+, but it should work with the free version if your dataset is small enougth. Due to space limits in colab it likely won't work with CV datasets that are bigger than 150 GB when extracted and converted to wav and converting and packing the files inside of colab gets very hard with datasets that are biger than 15 GB.




# Basic setup

In [1]:
## Install Coqui STT 
!git clone --depth 1 https://github.com/coqui-ai/STT.git
!cd STT; pip install -U pip wheel setuptools; pip install .
!pip uninstall tensorflow && pip install tensorflow-gpu==1.15.4

Cloning into 'STT'...
remote: Enumerating objects: 2211, done.
remote: Counting objects: 100% (2211/2211), done.
remote: Compressing objects: 100% (1396/1396), done.
remote: Total 2211 (delta 860), reused 1720 (delta 716), pack-reused 0
Receiving objects: 100% (2211/2211), 12.53 MiB | 24.43 MiB/s, done.
Resolving deltas: 100% (860/860), done.
     |████████████████████████████████| 1.7 MB 3.8 MB/s 
     |████████████████████████████████| 946 kB 66.1 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you 

In [2]:
# install libraries to convert mp3 to wav
!apt-get install sox libsox-fmt-mp3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libid3tag0 libmad0 libmagic-mgc libmagic1 libopencore-amrnb0
  libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libid3tag0 libmad0 libmagic-mgc libmagic1 libopencore-amrnb0
  libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox-fmt-mp3 libsox3
  sox
0 upgraded, 11 newly installed, 0 to remove and 37 not upgraded.
Need to get 872 kB of archives.
After this operation, 7,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:

## ✅ Mount Google Drive and Download your alphabet.txt

**First things first**: we need some data from Google Drive, GitHub or another source of your choice. 


In [3]:
# mount your private google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# create folder
! mkdir -p /content/eo/
%cd /content/eo/
! wget https://raw.githubusercontent.com/parolrekonado/deepspeech-esperanto/master/alphabet.txt

/content/eo
--2021-11-06 22:40:34--  https://raw.githubusercontent.com/parolrekonado/deepspeech-esperanto/master/alphabet.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 339 [text/plain]
Saving to: ‘alphabet.txt’

alphabet.txt        100%[===================>]     339  --.-KB/s    in 0s      

2021-11-06 22:40:34 (17.3 MB/s) - ‘alphabet.txt’ saved [339/339]



## Convert mp3s to wav and create a tar.gz file of it
**You only have to do this once, after that skip this step and use the tar file.** If you can do this on a local machine, I recommend not doing this on Colab and simply upload the result to google drive and skip this step.

Based on https://stt.readthedocs.io/en/latest/COMMON_VOICE_DATA.html?highlight=common%20voice

In [ ]:
# untar the Dataset from Common Voice
!mkdir -p /content/data
!tar -xzvf "/content/drive/MyDrive/Deepspeech/cv-corpus-7.0-2021-07-21-eo.tar.gz" -C "/content/data"   

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416530.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416542.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416543.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416544.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416545.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416546.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416552.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416553.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416554.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416555.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416556.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416562.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416563.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_27416564.mp3
cv-corpus-7.0-2021-07-21/eo/clips/common_voice_eo_2

In [ ]:
#rename folder for easier paths below
!mv /content/data/cv-corpus-7.0-2021-07-21 /content/data/cv-corpus-7

In [ ]:
# This step converts the mp3s to wav-files. The result will be around three times as big as your mp3 folder.
!/content/STT/bin/import_cv2.py --filter_alphabet /content/eo/alphabet.txt /content/data/cv-corpus-7/eo --normalize


Loading TSV file:  /content/data/cv-corpus-7/eo/test.tsv
Importing mp3 files...
Progress |######################################################| 100% completedImported 14420 samples.
Skipped 2 samples that failed on transcript validation.
Skipped 1 samples that were too short to match the transcript.
Skipped 240 samples that were longer than 10 seconds.
Final amount of imported audio: 23:11:34 from 23:52:43.
Saving new Coqui STT-formatted CSV file to:  /content/data/cv-corpus-7/eo/clips/test.csv
Writing CSV file for train.py as:  /content/data/cv-corpus-7/eo/clips/test.csv
Progress |######################################################| 100% completed
Loading TSV file:  /content/data/cv-corpus-7/eo/dev.tsv
Importing mp3 files...
Progress |##################################################### |  98% completedImported 14484 samples.
Skipped 3 samples that failed on transcript validation.
Skipped 180 samples that were longer than 10 seconds.
Final amount of imported audio: 22:45:03 from

In [ ]:
#delete all mp3 files AFTER they got converted to wav (rm doesnt work with so many files in Colab)
!find /content/data/cv-corpus-7/eo/clips/ -name "*.mp3" -delete

In [ ]:
#pack WAVs and CSVs into tar.gz inside of the workspace
!tar czf /content/data/converted-eo-corpus-7.tar.gz /content/data/cv-corpus-7/

tar: Removing leading `/' from member names


In [ ]:
#copy big file to Google Drive (for small files in low numbers mv also works well). 
#If space gets low during the transfere open the terminal and use "find /content/data/cv-corpus-7/eo/clips/ -name "*.wav" -delete"
import shutil
shutil.move("/content/data/converted-eo-corpus-7.tar.gz", "/content/drive/MyDrive/Deepspeech/")

'/content/drive/MyDrive/Deepspeech/converted-eo-corpus-7.tar.gz'

In [ ]:
# if files don't appear in your Google Drive, this often helps (sync and disconnect drive)
from google.colab import drive
drive.flush_and_unmount()

# Untar converted wav-files and download checkpoints
This part uses the prepered file from above. This speeds up the process a lot, untaring is a lot quicker then converting everything every time you want to train a model

In [ ]:
! mkdir -p /content/data
!tar -xzvf "/content/drive/MyDrive/Deepspeech/converted-eo-corpus-7.tar.gz" -C "/content/data"   

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
content/data/cv-corpus-7/eo/clips/common_voice_eo_25424494.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25365840.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25317120.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_27399051.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_22978904.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25322963.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25362935.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25606051.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25449421.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25235163.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25622072.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_24677607.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_25117525.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_24956391.wav
content/data/cv-corpus-7/eo/clips/common_voice_eo_2

In [ ]:
#create some missing folders
! mkdir -p /content/eo/checkpoints
! mkdir -p /content/eo/exports

In [ ]:
#If you have some checkpoints from earlier trainings, you can import them here
import shutil
shutil.move("/content/drive/MyDrive/Deepspeech/cp/checkpoints", "/content/eo/checkpoints")

## ✅ Configure & set hyperparameters

Coqui STT comes with a long list of hyperparameters you can tweak. We've set default values, but you will often want to set your own. You can use `initialize_globals_from_args()` to do this. 

You must **always** configure the paths to your data, and you must **always** configure your alphabet. Additionally, here we show how you can specify the size of hidden layers (`n_hidden`), the number of epochs to train for (`epochs`), and to initialize a new model from scratch (`load_train="init"`).

In [7]:
from coqui_stt_training.util.config import initialize_globals_from_args

initialize_globals_from_args(
    alphabet_config_path="/content/eo/alphabet.txt",
    train_files=["/content/data/content/data/cv-corpus-7/eo/clips/train-all.csv"],
    dev_files=["/content/data/content/data/cv-corpus-7/eo/clips/dev.csv"],
    test_files=["/content/data/content/data/cv-corpus-7/eo/clips/test.csv"],
    load_train="init",
    n_hidden=100,
    epochs=5,
    train_batch_size=32,
    dev_batch_size=32,
    test_batch_size=32,
    export_batch_size=32,
    automatic_mixed_precision=True,
    checkpoint_dir="/content/eo/checkpoints",
    export_dir="/content/eo/exports"
)

In [8]:
from coqui_stt_training.util.config import Config

# Take a peek at the entire Config
print(Config.to_json())

{
    "train_files": [
        "/content/data/content/data/cv-corpus-7/eo/clips/train-all.csv"
    ],
    "dev_files": [
        "/content/data/content/data/cv-corpus-7/eo/clips/dev.csv"
    ],
    "test_files": [
        "/content/data/content/data/cv-corpus-7/eo/clips/test.csv"
    ],
    "metrics_files": [],
    "auto_input_dataset": "",
    "vocab_file": "",
    "read_buffer": 1048576,
    "feature_cache": "",
    "cache_for_epochs": 0,
    "shuffle_batches": false,
    "shuffle_start": 1,
    "shuffle_buffer": 1000,
    "feature_win_len": 32,
    "feature_win_step": 20,
    "audio_sample_rate": 16000,
    "normalize_sample_rate": true,
    "augment": null,
    "epochs": 5,
    "dropout_rate": 0.05,
    "dropout_rate2": 0.05,
    "dropout_rate3": 0.05,
    "dropout_rate4": 0.0,
    "dropout_rate5": 0.0,
    "dropout_rate6": 0.05,
    "relu_clip": 20.0,
    "beta1": 0.9,
    "beta2": 0.999,
    "epsilon": 1e-08,
    "learning_rate": 0.001,
    "train_batch_size": 64,
    "dev_batch_

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀 (using the configure you set above).

This notebook should work on either a GPU or a CPU. However, GPU training is a lot quicker.

In [9]:
from coqui_stt_training.train import train, early_training_checks

early_training_checks()

train()

I Enabling automatic mixed precision training.
I Initializing all variables.
I STARTING Optimization
Epoch 0 |   Training | Elapsed Time: 0:00:00 | Steps: 0 | Loss: 0.000000       
Epoch 0 | Validation | Elapsed Time: 0:00:00 | Steps: 0 | Loss: 0.000000 | Dataset: /content/data/content/data/cv-corpus-7/eo/clips/dev.csv

UnknownError: ignored

## ✅ Test the model


In [ ]:
from coqui_stt_training.evaluate  import test

test()

# export model or checkpoints to Drive

In [ ]:
#copy big file to Google Drive (for small files in low numbers mv also works well). 
#If space gets low during the transfere open the terminal and use "find /content/data/ -name "*.wav" -delete"
import shutil
shutil.move("/content/eo", "/content/drive/MyDrive/Deepspeech/cp/")

In [ ]:
# if files don't appear in your Google Drive, this often helps (sync and disconnect drive)
from google.colab import drive
drive.flush_and_unmount()